Due date: October 21 2022 

# Description
  In this problem we will use apply the LSMC method to price American put options. Specifically, we will replicate the result in the first row, 6th column of Table 1 in [Longstaff and Schwartz 2001](https://www.anderson.ucla.edu/documents/areas/fac/finance/least_squares.pdf)

  

*  Read the introduction of the [paper](https://www.anderson.ucla.edu/documents/areas/fac/finance/least_squares.pdf).
*   We will price an american put option as described in page 126 of the aforementioned article. Read paragraphs 1 and 2 of page 126
* As we saw in class, one of the ways we can use linear regression to fit nonlinear functions is to use polynomial features. A common choice in many applications is to use the so called ``Chebyshev polynomials''. Chebyshev polynomials are defined recursively by:

\begin{equation}
T_0(x) = 1\\
T_1(x) = x\\
T_{n + 1}(x)  =  2 x T_n(x) - T_{n - 1}(x)\\
\end{equation}


# Group Member:

Chang Gao (UIN: 671189915)

Hyoung Woo Hahm (UIN: 661790448)

Napaton Prasertthum (UIN: 675585068)

Saranpat Prasertthum (UIN: 655667271)

Yu-Ching Liao (UIN: 656724372)

# Part 1
The code below simulates the evolution of a stock price that follows a geometric brownian motion. Write a JAX version of that code. You are not allowed to use functions from other libraries. For this part, the "simulate"
function does not need to be jit compiled. As we will see, jit compiling a funciton with for loops may introduce some complications
 

In [2]:
import numpy as np

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100


def simulate():
    np.random.seed(0)

    def step(S):
        dZ = np.random.normal(size=S.size) * np.sqrt(dt)
        dS = r * S * dt + σ * S * dZ
        S = S + dS
        return S

    S0 = np.ones(20000)
    S = S0
    S_list = []

    for t in range(m):
        S = step(S)
        S_list.append(S)

    S_array = np.stack(S_list)
    return S_array
print(simulate())

[[1.00715621 1.00170063 1.00401495 ... 0.99987096 0.99586427 0.99879389]
 [1.00858656 1.00179888 1.00740096 ... 1.00672951 0.99676042 0.99400235]
 [1.0008628  1.00485977 1.01312172 ... 1.01043822 0.98842005 0.99337036]
 ...
 [1.00834903 0.96949369 0.96891127 ... 1.06609344 0.98978317 1.00386936]
 [1.0045846  0.97349649 0.96590568 ... 1.06415365 0.98488869 1.00491575]
 [1.00652461 0.97005164 0.96454623 ... 1.05651341 0.98368471 1.00345296]]


# Solution to Part 1

In [6]:
# JAX version
import jax.numpy as jnp
import jax

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100


def simulate1():
    rng = jax.random.PRNGKey(0)

    def step(rng, S):
        dZ = jax.random.normal(rng, shape=(S.size,)) * jnp.sqrt(dt)
        dS = r * S * dt + σ * S * dZ
        S = S + dS
        return S

    S0 = jnp.ones(20000)
    S = S0
    S_list = []

    for t in range(m):
        rng, _ = jax.random.split(rng)
        S = step(rng, S)
        S_list.append(S)

    S_array = jnp.stack(S_list)
    return S_array
print(simulate1())
print(simulate1()[0].shape)

[[1.0031239  0.99879605 1.0028076  ... 1.001435   1.0056727  1.0059313 ]
 [1.0056242  0.9989038  1.0038857  ... 0.99900013 1.0030652  0.9993646 ]
 [1.0030042  0.9944063  1.0047244  ... 0.9981365  1.0078664  0.996579  ]
 ...
 [1.0453889  0.9646973  1.0070413  ... 0.9665726  1.0149897  1.0383427 ]
 [1.0339493  0.9706413  1.0091043  ... 0.96072483 1.0120413  1.036808  ]
 [1.0337375  0.9659554  1.0131707  ... 0.96760684 1.0132614  1.0382613 ]]
(20000,)


# Part 2
Write a jit compiled version of the simulate function. You may want to check out the function jax.lax.scan.


# Solution to Part 2

In [ ]:
# Jit compiled version
import jax.numpy as jnp
import jax

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100


@jax.jit
def simulate2():
    rng = jax.random.PRNGKey(0)

    def step(S, x):
        dZ = x * jnp.sqrt(dt)
        dS = r * S * dt + σ * S * dZ
        S = S + dS
        return S, S

    S0 = jnp.ones(20000)
    init = jnp.zeros(20000)
    xs = jax.random.normal(rng, shape=(m, S0.size))
    carry, S_array = jax.lax.scan(step, S0, xs)

    return S_array

In [ ]:
# test
# Original simulate function
# simulate()

# JAX version
# simulate1()

# Jit compiled version
# simulate2()

# Part 3
The code below computes the price of an American Put option using Least Squares Monte Carlo (LSMC). Write a JAX version of that code. You are not allowed to use functions from other libraries. Your "compute_price" function must be jit compiled.

In [ ]:
import numpy as np

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 12   # Polynmial order
Δt = T / m  # interval between two exercise dates


# Construct polynomial features of order up to k using the
# recursive formulation
def chebyshev_basis(x, k):
    B = [np.ones(len(x)), x]
    for n in range(2, k):
        Bn = 2 * x * B[n - 1] - B[n - 2]
        B.append(Bn)

    return np.column_stack(B)


# scales x to be in the interval(-1, 1)
def scale(x):
    xmin = x.min()
    xmax = x.max()
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b


# simulates one step of the stock price evolution
def step(S):
    dB = np.sqrt(Δt) * np.random.normal(size=S.size)
    S_tp1 = S + r * S * Δt + σ * S * dB
    return S_tp1


def payoff_put(S):
    return np.maximum(K - S, 0.)


# LSMC algorithm
def compute_price():
    np.random.seed(0)
    S0 = Spot * np.ones(n)
    S = [S0]

    for t in range(m):
        S_tp1 = step(S[t])
        S.append(S_tp1)

    discount = np.exp(-r * Δt)

    # Very last date
    value_if_exercise = payoff_put(S[-1])
    discounted_future_cashflows = value_if_exercise * discount

    # Proceed recursively
    for i in range(m - 1):
        X = chebyshev_basis(scale(S[-2 - i]), order)
        Y = discounted_future_cashflows

        Θ = np.linalg.solve(X.T @ X, X.T @ Y)
        value_if_wait = X @ Θ
        value_if_exercise = payoff_put(S[-2 - i])
        exercise = value_if_exercise >= value_if_wait
        discounted_future_cashflows = discount * np.where(
            exercise,
            value_if_exercise,
            discounted_future_cashflows)

    return discounted_future_cashflows.mean()


print(compute_price())
# test = compute_price(order, Spot, σ, K, r)

4.4634476148734405


# Solution to Part 3

In [9]:
# JAX version
import jax.numpy as jnp
import jax

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 12   # Polynmial order
Δt = T / m  # interval between two exercise dates


# Construct polynomial features of order up to k using the
# recursive formulation
def chebyshev_basis(x, k):

    def Bn(carry, xs):
        b1, b2 = carry
        Bn = 2 * x * b2 - b1
        carry = b2, Bn
        return carry, Bn

    init = jnp.ones(len(x)), x
    carry, B = jax.lax.scan(Bn, init, xs=None, length=k-2)

    B = jnp.insert(B, 0, values=x, axis=0)
    B = jnp.insert(B, 0, values=jnp.ones(len(x)), axis=0)

    return B.T


# scales x to be in the interval(-1, 1)
def scale(x):
    xmin = x.min()
    xmax = x.max()
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b


# simulates one step of the stock price evolution
def step(S, x):
    dB = jnp.sqrt(Δt) * x
    S_tp1 = S + r * S * Δt + σ * S * dB
    return S_tp1, S_tp1


def payoff_put(S):
    return jnp.maximum(K - S, 0.)


# LSMC algorithm
@jax.jit
def compute_price():
    rng = jax.random.PRNGKey(0)
    S0 = Spot * jnp.ones(n)
    xs = jax.random.normal(rng, shape=(m, S0.size))
    carry, S = jax.lax.scan(step, S0, xs)
    S = jnp.insert(S, 0, values=S0, axis=0)

    discount = jnp.exp(-r * Δt)

    # Very last date
    value_if_exercise = payoff_put(S[-1])
    discounted_future_cashflows = value_if_exercise * discount

    def recur(Y, S):
        X = chebyshev_basis(scale(S), order)
        Θ = jnp.linalg.solve(X.T @ X, X.T @ Y)
        value_if_wait = X @ Θ
        value_if_exercise = payoff_put(S)
        exercise = value_if_exercise >= value_if_wait
        Y = discount * jnp.where(
            exercise,
            value_if_exercise,
            Y)
        return Y, Y

    Y = discounted_future_cashflows
    discounted_future_cashflows, ys = jax.lax.scan(
        recur, Y, S[:-1], reverse=True)

    return discounted_future_cashflows.mean()


print(compute_price())
# test = compute_price(order, Spot, σ, K, r)

4.4773197
